# Load Libs

In [8]:
import os
# os.environ['PYUSB_DEBUG'] = 'debug'

try:
    import libusb1
except ModuleNotFoundError:
    !pip3 install libusb1
    import usb1
    
try:
    import libusb
except ModuleNotFoundError:
    !pip3 install --upgrade libusb
    !pip3 install --upgrade tox
    import libusb

try:
    import usb.core
    import usb.util
    import usb.control
except ModuleNotFoundError:
    !pip3 install --upgrade pyusb
    import usb.core
    import usb.control
    
try:
    import hid
except ModuleNotFoundError:
    !pip3 install --upgrade hidapi
    import hid


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 KB 1.0 MB/s eta 0:00:00a 0:00:01
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


## USB list all existing devices

In [28]:
printers = usb.core.find(find_all=True)
for printer in printers:
    print(printer)

DEVICE ID 043e:9a39 on Bus 000 Address 002 =================
 bLength                :   0x12 (18 bytes)
 bDescriptorType        :    0x1 Device
 bcdUSB                 :  0x200 USB 2.0
 bDeviceClass           :   0xef Miscellaneous
 bDeviceSubClass        :    0x2
 bDeviceProtocol        :    0x1
 bMaxPacketSize0        :   0x40 (64 bytes)
 idVendor               : 0x043e
 idProduct              : 0x9a39
 bcdDevice              :  0x411 Device 4.11
 iManufacturer          :    0x1 LG Electronics Inc.
 iProduct               :    0x3 LG Monitor Controls
 iSerialNumber          :    0x4 201NTCZD5427
 bNumConfigurations     :    0x1
  CONFIGURATION 1: 0 mA ====================================
   bLength              :    0x9 (9 bytes)
   bDescriptorType      :    0x2 Configuration
   wTotalLength         :   0x6b (107 bytes)
   bNumInterfaces       :    0x3
   bConfigurationValue  :    0x1
   iConfiguration       :    0x0 
   bmAttributes         :   0xc0 Self Powered
   bMaxPower       

libusb: warning [darwin_open] USBDeviceOpen: another process has device opened for exclusive access


## Load our device
We are looking for vendor-id 0x043e (LG) and product-id 0x9a39 (32UN880).

In [3]:
lgMon = usb.core.find(idVendor=0x043e, idProduct=0x9a39)
if lgMon is None:
    raise ValueError('failed to find the monitor')
print(f'found monitor "{lgMon.serial_number}"')

lgMon.set_configuration()



found monitor "201NTCZD5427"


# Enable debugging

In [4]:
lgMon.backend.lib.libusb_set_debug(lgMon.backend.ctx, 3)

3

# Read out current brightness

LIBUSB_RECIPIENT_INTERFACE = 0x01
LIBUSB_REQUEST_TYPE_CLASS = 0x20
LIBUSB_ENDPOINT_OUT = 0x00


In [5]:
# Maybe the hidInterface and outputEndpoint is useless
#outputEndpoint = usb.util.find_descriptor(
#    hidInterface,
#    # match the first OUT endpoint
#    custom_match = lambda e: \
#        usb.util.endpoint_direction(e.bEndpointAddress) == usb.util.ENDPOINT_OUT)
#if outputEndpoint is None:
#    raise ValueError('failed to find output endpoint of given interface')
#print('found output endpoint')


In [7]:
msg = '1234567'
requestType = 0x21 
request = 0x09 # HID set_report
lgMon.ctrl_transfer(bmRequestType = 0x21 # LIBUSB_REQUlibusb_unref_deviceEST_TYPE_CLASS|LIBUSB_RECIPIENT_INTERFACE|LIBUSB_ENDPOINT_OUT
                    , bRequest = 0x09    # HID set_report/
                    , wValue = 3 << 8    # HID feature
                    , wIndex = 0
                    , data_or_wLength = 7)


libusb: info [darwin_claim_interface] USBInterfaceOpen: another process has device opened for exclusive access


USBError: [Errno 13] Access denied (insufficient permissions)

In [11]:
h = hid.device()
h.open(vendor_id=0x043e, product_id=0x9a39)

In [22]:
h.set_nonblocking(False)
h.get_serial_number_string()

'201NTCZD5427'

In [27]:
h.get_feature_report(0, 6)

OSError: read error

In [25]:
h.error()

'hid_error is not implemented yet'